## This file helps to Build and test the LLama model and Langchain functions

In [1]:
# Importing packages

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

In [2]:
#setting up env
load_dotenv()

True

In [3]:
#Building LLM
llm = ChatGroq(
    model='llama3-8b-8192',
    groq_api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.5
    )

In [6]:
#Getting response 
response = llm.invoke("Tell me about Karim Nagar in Telangana")
print(response)

content="Karimnagar is a city and the district headquarters of Karimnagar district in the Indian state of Telangana. It is located about 170 km north of the state capital, Hyderabad. Here are some key facts about Karimnagar:\n\n1. History: Karimnagar has a rich history dating back to the 12th century. It was a major center of power during the Kakatiya dynasty, and later it was a part of the Vijayanagara Empire.\n2. Geography: Karimnagar is situated on the banks of the Manair River, a tributary of the Godavari River. The city is surrounded by the Deccan Plateau and is known for its fertile soil and abundant water resources.\n3. Economy: Karimnagar is an important agricultural hub, with crops like paddy, cotton, and sugarcane being major cash crops. The city is also known for its handloom industry, with traditional textile products like silk and cotton fabrics being produced.\n4. Demographics: As of the 2011 census, Karimnagar has a population of around 300,000 people. The city has a mix

In [10]:
from langchain.document_loaders import WebBaseLoader

url = "https://www.google.com/"
loader = WebBaseLoader(url)
docs = loader.load()

print(docs[0].page_content, end = ' ')  # Extracted text from the webpage

GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in Advanced searchAdvertisingBusiness SolutionsAbout Google© 2025 - Privacy - Terms  